In [1]:
import numpy as np
import pandas as pd
import seaborn
import matplotlib.pyplot as plt
import random
from sklearn.model_selection import KFold

In [2]:
with open('data/X_train.npy', 'rb') as f:
    X_train = np.load(f)

In [3]:
path_y_train = 'data/y_train.csv'

y_train = pd.read_csv(path_y_train, index_col=0)
y_train = y_train.values.flatten()

In [4]:
from sklearn.metrics import roc_auc_score


def gini(auc_score):
    return 2 * auc_score - 1

In [5]:
np.unique(y_train, return_counts=True)[1]/y_train.shape[0]

array([0.80697097, 0.19302903])

In [6]:
# from imblearn.over_sampling import SMOTE

# sm = SMOTE(random_state=42)
# X_res, y_res = sm.fit_resample(X_train, y_train)

In [7]:
# from imblearn.over_sampling import ADASYN

# sm = ADASYN(random_state=0, n_neighbors=)
# X_res, y_res = sm.fit_resample(X_train, y_train)

In [8]:
# np.unique(y_res, return_counts=True)[1]/y_train.shape[0]

In [9]:
from sklearn.model_selection import train_test_split


X_train_preprocess2, X_valid_preprocess2, y_train_preprocess2, y_valid_preprocess2 =\
      train_test_split(X_train, y_train, test_size=0.33, random_state=100)

In [10]:
from sklearn.utils.class_weight import compute_class_weight


class_weights = compute_class_weight(class_weight="balanced",
                                     classes=np.unique(y_train),
                                     y=y_train)

In [27]:
from catboost import CatBoostClassifier


params = {'iterations': 2000,
          # 'learning_rate': 0.015,
          'depth': 15,
          # 'l2_leaf_reg': 0.01313,
          'loss_function': 'Logloss',
          'eval_metric': 'NormalizedGini', # 'AUC'
          'score_function': 'L2', # 'Cosine', 'L2', 'NewtonCosine', 'NewtonL2'
          'bagging_temperature': 3,
          # 'scale_pos_weight': class_weights[1],
          'class_weights': class_weights,
          # 'auto_class_weights': 'Balanced', # 'SqrtBalanced'
          'grow_policy': 'Lossguide',   # 'Depthwise', 'Lossguide', 'SymmetricTree'
          'bootstrap_type': 'Bayesian', # 'Bernoulli', 'Bayesian'
          'early_stopping_rounds': 200,
        #   'cat_features': cat_features,
          'task_type': 'GPU',
          'border_count': 254,  # 254
          'verbose': 200,
          'random_seed': 0,
        #   'ignored_features': ignored_features
         }

# params ={'depth': 7,
#  'l2_leaf_reg': 0.011294834927782476,
#   'bagging_temperature': 9.297813770415363,
#    'auto_class_weights': 'Balanced',
# # 'scale_pos_weight': 5,
#     'grow_policy': 'Lossguide',
#      'min_data_in_leaf': 10,
#       'max_leaves': 41,
#       'verbose': 200,
#       'task_type': 'GPU',
#       'border_count': 254,
#       'eval_metric': 'NormalizedGini',}


# cbc = CatBoostClassifier(**params)
# kf = KFold(n_splits=5, shuffle=True)
# gini_test = []
# for train_index, test_index in kf.split(X_res):
#     x_train_fold, x_test_fold = X_res[train_index], X_res[test_index]
#     y_train_fold, y_test_fold = y_res[train_index], y_res[test_index]

#     cbc.fit(x_train_fold, y_train_fold,
#             eval_set=(x_test_fold, y_test_fold))

#     # pred_proba = cbc.predict_proba(x_test_fold)[:,1]
#     pred = cbc.predict(x_test_fold)
#     gini_test.append(gini(roc_auc_score(y_test_fold, pred)))

cbc = CatBoostClassifier(**params)
cbc.fit(X_train_preprocess2, y_train_preprocess2,
        eval_set=(X_valid_preprocess2, y_valid_preprocess2))
# 0.7109
# 0.867

Learning rate set to 0.030266
0:	test: 0.3872493	best: 0.3872493 (0)	total: 73.8ms	remaining: 2m 27s


Default metric period is 5 because NormalizedGini is/are not implemented for GPU
Metric NormalizedGini is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


200:	test: 0.4415529	best: 0.4415529 (200)	total: 14.6s	remaining: 2m 10s
400:	test: 0.4481012	best: 0.4481012 (400)	total: 29s	remaining: 1m 55s
600:	test: 0.4506274	best: 0.4506274 (600)	total: 43.5s	remaining: 1m 41s
800:	test: 0.4520328	best: 0.4520328 (800)	total: 58s	remaining: 1m 26s
1000:	test: 0.4531253	best: 0.4531253 (1000)	total: 1m 12s	remaining: 1m 12s
1200:	test: 0.4539907	best: 0.4539907 (1200)	total: 1m 26s	remaining: 57.7s
1400:	test: 0.4546377	best: 0.4546377 (1400)	total: 1m 41s	remaining: 43.3s
1600:	test: 0.4551463	best: 0.4551463 (1600)	total: 1m 55s	remaining: 28.8s
1800:	test: 0.4556126	best: 0.4556126 (1800)	total: 2m 10s	remaining: 14.4s
1999:	test: 0.4560196	best: 0.4560196 (1999)	total: 2m 24s	remaining: 0us
bestTest = 0.4560195984
bestIteration = 1999


In [28]:
pred = cbc.predict(X_valid_preprocess2)
gini(roc_auc_score(y_valid_preprocess2.flatten(), pred))
# 0.583524 best

0.33099534197869684

In [17]:
pred = cbc.predict(X_train)
gini(roc_auc_score(y_train.flatten(), pred))
# 0.38896 - 0.257
# 0.43278 best

0.4316006575874296

In [79]:
0.4267585 - 0.13195999999999997

0.2947985

In [22]:
params = {'iterations': 1919,
          # 'learning_rate': 0.015,
          'depth': 7,
          'l2_leaf_reg': 0.01313,
          'loss_function': 'Logloss',
          'eval_metric': 'NormalizedGini', # 'AUC'
          'score_function': 'L2', # 'Cosine', 'L2', 'NewtonCosine', 'NewtonL2'
          # 'bagging_temperature': 3,
          # 'scale_pos_weight': class_weights[1],
          'class_weights': class_weights,
          # 'auto_class_weights': 'Balanced', # 'SqrtBalanced'
          'grow_policy': 'SymmetricTree',   # 'Depthwise', 'Lossguide', 'SymmetricTree'
          'bootstrap_type': 'Bernoulli', # 'Bernoulli', 'Bayesian'
          'early_stopping_rounds': 200,
        #   'cat_features': cat_features,
          'task_type': 'GPU',
          'border_count': 254,  # 254
          'verbose': 200,
          'random_seed': 0,
        #   'ignored_features': ignored_features
         }

cbc = CatBoostClassifier(**params)
cbc.fit(X_train, y_train)

Default metric period is 5 because NormalizedGini is/are not implemented for GPU
Metric NormalizedGini is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	total: 50.3ms	remaining: 1m 36s
200:	total: 6.25s	remaining: 53.4s
400:	total: 12.1s	remaining: 46s
600:	total: 17.7s	remaining: 38.9s
800:	total: 23.4s	remaining: 32.6s
1000:	total: 29s	remaining: 26.6s
1200:	total: 34.6s	remaining: 20.7s
1400:	total: 40.4s	remaining: 14.9s
1600:	total: 46s	remaining: 9.13s
1800:	total: 51.7s	remaining: 3.39s
1918:	total: 55.1s	remaining: 0us


In [23]:
pred = cbc.predict(X_train)
gini(roc_auc_score(y_train.flatten(), pred))


0.36184410704182945

In [14]:
from sklearn.metrics import classification_report
from sklearn.metrics import (accuracy_score , f1_score , precision_score , recall_score)


print(classification_report(y_train.flatten() , pred))

              precision    recall  f1-score   support

           0       0.90      0.66      0.76    968253
           1       0.33      0.70      0.44    231608

    accuracy                           0.66   1199861
   macro avg       0.61      0.68      0.60   1199861
weighted avg       0.79      0.66      0.70   1199861



In [75]:
from sklearn.metrics import classification_report
from sklearn.metrics import (accuracy_score , f1_score , precision_score , recall_score)


print(classification_report(y_train.flatten() , pred))

              precision    recall  f1-score   support

           0       0.92      0.68      0.78    968253
           1       0.36      0.74      0.48    231608

    accuracy                           0.69   1199861
   macro avg       0.64      0.71      0.63   1199861
weighted avg       0.81      0.69      0.72   1199861



In [19]:
with open('data/X_test.npy', 'rb') as f:
    X_test = np.load(f)

In [20]:
test_pred = cbc.predict(X_test)

In [21]:
submit = pd.read_csv('data/answer.csv')
submit['loan_status'] = test_pred

In [22]:
submit.loan_status.value_counts()/submit.loan_status.shape[0]

0    0.614733
1    0.385267
Name: loan_status, dtype: float64

In [23]:
submit.to_csv('answer.csv')

In [24]:
import zipfile
      
jungle_zip = zipfile.ZipFile('answer.zip', 'w')
jungle_zip.write('answer.csv', compress_type=zipfile.ZIP_DEFLATED)
jungle_zip.close()